# Fase 1.2: Convertir la ResBaGAN inicial de PyTorch a ONXX

In [ ]:
# Imports

import math,random,struct,os,time,sys
import numpy as np
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime as ort
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from sklearn import preprocessing
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time

# Funciones y parámetros de la ResBaGAN
import sys
sys.path.append('../src/')

import resbagan_networks
import resbagan_datasets
from cnn21_pix import read_pgm, save_pgm

## 1. Conversión de la ResBaGAN inicial de PyTorch a ONNX

### 1.1. Conversión del modelo

In [ ]:
# Datos de entrada

batch_size = 100
B = 5
sizex = 32
sizey = 32

In [ ]:
# Definir parámetros y cargar datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'
MODEL="../results/models/model_ResBaGAN.pt"

# Queremos usar todos los datos para la inferencia
SAMPLES=[0.0,0.0]

# Carga de datos para la inferencia en el discriminador
dataset = resbagan_datasets.HyperDataset(
    "oitaven_river", segmented=False, patch_size=32, ratios=(SAMPLES[0], SAMPLES[1]))

# Almacenamos las dimensiones en variables
H = dataset.height
V = dataset.width

samples = dataset.ordered_test_set['samples']

# Obtenemos el array de gt
truth = dataset.gt.flatten()
# Obtenemos un array de indices para test
test = dataset.test_index_list

# Obtenemos el numero de clases y el numero de clases no vacias para test
nclases = dataset.classes_count
nclases_no_vacias = 0
for i in range(nclases):
    clase_actual = i + 1
    if any(truth[idx] == clase_actual for idx in test):
        nclases_no_vacias += 1

print(dataset)

# The custom HyperDataset object contains all the train, validation and test data
#   --> But it will wrapped into a PyTorch data feeder for convenience
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=100,
    shuffle=False,
    num_workers=4,
    pin_memory=False,
)

In [ ]:
# Cargar el modelo

cuda=True if torch.cuda.is_available() else False
device=torch.device('cuda' if cuda else 'cpu')
device='cpu'

if torch.backends.cudnn.is_available():
    print('* Activando CUDNN')
    torch.backends.cudnn.enabled=True
    torch.backends.cudnn.beBhmark=True

hyperparams = {
    "latent_size":   128,
    "activation":    "lrelu",
    "p_dropout":     0.05,
    "weight_init":   "xavier",
    "learning_rate": 0.001,
    "epochs":        100,
    "batch_size":    100,
    "num_workers":   4,
    "device":        "cpu",
}

# Cargar el discriminador
model = resbagan_networks.ResBaGAN_Discriminator(dataset=dataset, device=hyperparams["device"], hyperparams=hyperparams)

# Cargar los pesos guardados
model.load_state_dict(torch.load("../results/models/model_ResBaGAN_discriminator.pt"))
model.eval()

In [ ]:
# Definimos una función personalizada para la operación torch.vdot
# Esto se debe a que el conversor no soporta esta operación

def vdot_replacement(a, b):
    return torch.sum(torch.conj(a) * b)

In [ ]:
# Asignamos la nueva operación torch.vdot a la función personalizada

torch.vdot = vdot_replacement

In [ ]:
# Convertir el modelo a ONNX

# Crear un tensor de entrada de ejemplo 
# El tensor tendrá tamanho (batch_size, canales, altura, ancho)
# Lo ajustamos al rango [-1, 1]
input_tensor = (torch.rand(batch_size, B, sizex, sizey) * 2 - 1).to(device)

# Exportamos el modelo a onnx
onnx_filename = "../results/models/model_ResBaGAN_discriminator.onnx"
torch.onnx.export(
    model, # Modelo entrenado
    input_tensor, # Entrada de ejemplo
    onnx_filename, # Ruta de salida del archivo ONNX
    export_params=True, # Exportar parámetros del modelo
    opset_version=12, # Versión de opset
    do_constant_folding=True, # Optimización de constantes
    input_names=['input'], # Nombre de la entrada
    output_names=['output'], # Nombre de la salida
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

In [ ]:
# Comprobar que la conversión es correcta

onnx_model = onnx.load(onnx_filename)
onnx.checker.check_model(onnx_model)

### 1.2. Evaluación del modelo en ONNX

In [ ]:
# Cargar los modelos

# Cargar el modelo convertido a ONNX
ort_session = ort.InferenceSession("../results/models/model_ResBaGAN_discriminator.onnx")

In [ ]:
# Comprobar la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = (torch.rand(1, B, sizex, sizey) * 2 - 1).to(device)

# Salida del modelo PyTorch
with torch.no_grad():
    output_torch, _ = model(input_tensor)

# Salida del modelo ONNX
output_onnx = ort_session.run(None, {'input': input_tensor.cpu().numpy()})[0]

# Comparar las diferencias
error = np.abs(output_torch - output_onnx).mean()
print(f'Error medio entre PyTorch y ONNX: {error}')

In [ ]:
# Carga de datos e inferencia completa para el modelo en ONNX

# Medir el tiempo de ejecución
start_time = time.time()

# Definir parámetros y cargar datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'
MODEL="../results/models/model_ResBaGAN.pt"

# Queremos usar todos los datos para la inferencia
SAMPLES=[0.0,0.0]

# Carga de datos para la inferencia en el discriminador
dataset = resbagan_datasets.HyperDataset(
    "oitaven_river", segmented=False, patch_size=32, ratios=(SAMPLES[0], SAMPLES[1]))

# Almacenamos las dimensiones en variables
H = dataset.height
V = dataset.width

samples = dataset.ordered_test_set['samples']

# Obtenemos el array de gt
truth = dataset.gt.flatten()
# Obtenemos un array de indices para test
test = dataset.test_index_list

# Obtenemos el numero de clases y el numero de clases no vacias para test
nclases = dataset.classes_count
nclases_no_vacias = 0
for i in range(nclases):
    clase_actual = i + 1
    if any(truth[idx] == clase_actual for idx in test):
        nclases_no_vacias += 1

print(dataset)

# The custom HyperDataset object contains all the train, validation and test data
#   --> But it will wrapped into a PyTorch data feeder for convenience
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=100,
    shuffle=False,
    num_workers=4,
    pin_memory=False,
)

output=np.zeros(H*V, dtype=np.uint8)

# Modo evaluación
dataset.to_test()

# Realizar la predicción
total=0
for batch_id, (inputs, labels, targets_pixel_level) in enumerate(data_loader):
    inputs_np=inputs.numpy()
    # Realizar la inferencia
    outputs = ort_session.run(None, {'input': inputs_np})
    outputs_discriminator = outputs[0]
    outputs_discriminator[:, dataset.classes_count] = -math.inf # Se deshabilita la clase fake para test
    predicted=np.argmax(outputs_discriminator, axis=1)
    
    for i in range(len(predicted)):
        output[test[total+i]]=np.uint8(predicted[i]+1)
    total+=labels.size(0)
    if(total%100000==0): print('  Test:',total,'/', len(dataset))

end_time = time.time()

print("Prediction time: {:.4f} seconds".format(end_time - start_time))

In [ ]:
# Guardar el output

np.save('../results/predictions/predictions_ResBaGAN_discriminator_onnx.npy', output)

In [ ]:
# Cargar el output

output = np.load('../results/predictions/predictions_ResBaGAN_discriminator_onnx.npy')

In [ ]:
# Evaluar el desempeño del modelo

# Precisiones a nivel de clase
correct=0; total=0; AA=0; OA=0
class_correct=[0]*(nclases+1)
class_total=[0]*(nclases+1)
class_aa=[0]*(nclases+1)

for i in test:
    if(output[i]==0 or truth[i]==0): continue
    total+=1; class_total[truth[i]]+=1
    if(output[i]==truth[i]):
          correct+=1
          class_correct[truth[i]]+=1
for i in range(1,nclases+1):
    if(class_total[i]!=0): class_aa[i]=100*class_correct[i]/class_total[i]
    else: class_aa[i]=0
    AA+=class_aa[i]
OA=100*correct/total; AA=AA/nclases_no_vacias

for i in range(1,nclases+1): print('  Class %02d: %02.02f'%(i,class_aa[i]))
print('* Accuracy (pixels) OA=%02.02f, AA=%02.02f'%(OA,AA))
print('  total:',total,'correct:',correct)

In [ ]:
# Guardar la salida

save_pgm(output,V,H,nclases,'../results/predictions/predictions_ResBaGAN_discriminator_onnx.pgm')

In [ ]:
# Mostrar la salida

OUTPUT='../results/predictions/predictions_ResBaGAN_discriminator_onnx.pgm'

(imagen_output, H, V) = read_pgm(OUTPUT)

# Convertir la lista a array y redimensionar
imagen_output = np.array(imagen_output, dtype=np.uint8).reshape(V, H)

# Mostrar la imagen
plt.imshow(imagen_output)
plt.show()